In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
pd.options.mode.chained_assignment = None 
from matplotlib import pyplot
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import pickle
import csv
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
pd.set_option('display.max_columns', None)

In [2]:
def Final_Clean(df, direction):
  
    df = df.loc[df['DIRECTION'] == direction]
    
    #covert to appropriate data types
    df['DAYOFSERVICE'] = df['DAYOFSERVICE'].astype('datetime64[ns]')
    df['ROUTEID'] = df['ROUTEID'].astype('category')
    df['LINEID'] = df['LINEID'].astype('category')

    #adding an hour column so that dataframe can be merged with weather dataframe 
    df['HOUR'] = pd.to_datetime(df['DEPARTURE_TIME_ACTUAL'])
    df['HOUR'] = df['HOUR'].dt.hour

    #Grouping the trips by day of service and trip ID and then 
    #sorting the dataframe according to progrnumber (Sequential position of the stop point in the trip)
    sorted_df = df.groupby(["DAYOFSERVICE","TRIPID"]).apply(lambda x: x.sort_values(["PROGRNUMBER"], ascending = True)).reset_index(drop=True)

    #Creating a feature for the duration of the journey 
    #this is done by taking the actual arrival time of the preceding row from the arrival time of the following row.
    sorted_df['JOURNEY_DURATION'] = sorted_df['ACTUALTIME_ARR'].shift(-1) - sorted_df['ACTUALTIME_ARR'] - sorted_df['DWELL_TIME']

    #creating the peak hour feature
    sorted_df['PEAK_HOUR'] = [1 if x >'07:00:00' and x <'10:00:00' or x >'16:00:00' and x <'19:00:00' else 0 for x in sorted_df['ARRIVAL_TIME_ACTUAL']]

    #creating a public holiday indicator feature 
    dates = pd.to_datetime(['2018-01-01','2018-03-17','2018-03-19','2018-03-30','2018-04-01','2018-04-02','2018-05-07',
     '2018-06-04','2018-08-06','2018-11-29','2018-12-24','2018-12-25','2018-12-26', '2018-12-31'])

    sorted_df['PUBLIC_HOLIDAY'] = sorted_df['DAYOFSERVICE'].isin(dates)
    sorted_df['PUBLIC_HOLIDAY'] *= 1    
    
    #deleting columns that won't be used
    del sorted_df['ARRIVAL_TIME_ACTUAL']
    del sorted_df['DEPARTURE_TIME_ACTUAL']
    del sorted_df['PLANNEDTIME_DEP']
    del sorted_df['PLANNEDTIME_ARR']

    #we want to get rid of all rows where the bus has reached the final destination of the trip
    #the journey duration of these rows is not representative of the actual journey time as 
    #this is the time spent in the bay before the next trip 

    #we create a column END_OF_TRIP that will be 1 for all in transit journeys
    #and negative for the final destination as it is the subtraction of the progress number of the current section
    #from the progress number of the next section of the trip 
    sorted_df['NEXT_STOPPOINTID'] = sorted_df['STOPPOINTID'].shift(-1)
    sorted_df['NEXT_PROGNUMBER']=sorted_df["PROGRNUMBER"].shift(-1)
    sorted_df['END_OF_TRIP'] = sorted_df['NEXT_PROGNUMBER'] - sorted_df['PROGRNUMBER']

    #as the final rows journey duration, NEXT_STOPPOINTID, NEXT_PROGNUMBER and END_OF_TRIP are null
    sorted_df = sorted_df.dropna(axis=0)
 
    #merging weather data 
    weather_df = pd.read_csv('~/data/Clean_Weather_2018.csv')
    weather_df['date'] = weather_df['date'].astype('datetime64[ns]')
    weather_df['weather_main'] = weather_df['weather_main'].astype('category')

    final_df = pd.merge(sorted_df, weather_df, how='left',left_on=['DAYOFSERVICE','HOUR'],right_on=['date','hour'])
 
    del final_df['VEHICLEID']
    del final_df['date']
    del final_df['hour']

    final_df['MONTH']= final_df['MONTH'].astype('category')
    final_df['DAY']= final_df['DAY'].astype('category')
    final_df['HOUR']= final_df['HOUR'].astype('category')
    final_df['STOPPOINTID']= final_df['STOPPOINTID'].astype('category')
    final_df['NEXT_STOPPOINTID']= final_df['NEXT_STOPPOINTID'].astype('int')
    final_df['NEXT_STOPPOINTID']= final_df['NEXT_STOPPOINTID'].astype('category')
    final_df['JOURNEY_DURATION']= final_df['JOURNEY_DURATION'].astype('int')

    #removing the row at the end of each trip where journey duration is not required.
    final_df = final_df.loc[final_df['END_OF_TRIP'] == 1]
    
    #remove negative journey times
    final_df = final_df[(final_df['JOURNEY_DURATION'] > 0)]
    
    #remove outliers that are 3 standard deviations above the mean.
    final_df=final_df[np.abs(final_df.JOURNEY_DURATION-final_df.JOURNEY_DURATION.mean()) <= (3*final_df.JOURNEY_DURATION.std())]

    return final_df

In [3]:
def Random_Forest(df):

    X = pd.DataFrame(df[['DAY','HOUR','MONTH','STOPPOINTID','NEXT_STOPPOINTID',
                           'PEAK_HOUR', 'wind_speed', 'temp', 'humidity', 'weather_main']])
    
    y = df.JOURNEY_DURATION

    X = pd.get_dummies(X)
    
    dummies = X.columns
    
    with open("/home/tomah/data/Model_Testing/RandomForestPickles/RandomForest_route_{}_d{}_headers.pkl".format(Route, direction), 'wb') as handle:
        pickle.dump(dummies, handle, pickle.HIGHEST_PROTOCOL)

    #test-train split 
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    
    params = {'n_estimators': 32,
     'min_samples_split': 10,
     'min_samples_leaf': 4,
     'max_features': 'sqrt',
     'max_depth': 70,
     'bootstrap': True}
    
    #creating and fitting the RF model and  Fit model on full dataset
    rfc = RandomForestRegressor(**params).fit(X_train, Y_train)
    
    with open("/home/tomah/data/Model_Testing/RandomForestPickles/RandomForest_Model_route_{}_d{}.pkl".format(Route, direction), 'wb') as handle:
        pickle.dump(rfc, handle, pickle.HIGHEST_PROTOCOL)

    #predictions from the training set
    y_pred_rf_train = rfc.predict(X_train)
    
    #predicitng from the test set 
    y_pred_rf = rfc.predict(X_test)
    
    #some evaluation metrics.
    print('Mean Absolute Error (train):', metrics.mean_absolute_error(Y_train, y_pred_rf_train))
    print('Mean Absolute Error (test):', metrics.mean_absolute_error(Y_test, y_pred_rf))
    print('Mean Percentage Error:', np.mean((Y_test - y_pred_rf)/Y_test))
    print('Mean Absolute Percentage Error:', metrics.mean_absolute_percentage_error(Y_test,y_pred_rf)) 
    print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred_rf))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred_rf)))
    print('R-square value:', r2_score(Y_test,y_pred_rf))
    
    Mean_abs_error_train = metrics.mean_absolute_error(Y_train, y_pred_rf_train)
    Mean_abs_error_test = metrics.mean_absolute_error(Y_test, y_pred_rf)
    Mean_Percentage_Error = np.mean((Y_test - y_pred_rf)/Y_test)
    Mean_Absolute_Percentage_Error= metrics.mean_absolute_percentage_error(Y_test,y_pred_rf)  
    Mean_squared_error =  metrics.mean_squared_error(Y_test, y_pred_rf)
    Root_Mean_sqaured_error = np.sqrt(metrics.mean_squared_error(Y_test, y_pred_rf))
    R_square_value =  r2_score(Y_test,y_pred_rf)

    
    with open('Random_Forest_Results_2.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([Route, direction, Mean_abs_error_train, Mean_abs_error_test,Mean_Percentage_Error,
                         Mean_Absolute_Percentage_Error, Mean_squared_error, Root_Mean_sqaured_error, R_square_value])

In [4]:
Trips_df = pd.read_csv("~/data/rt_trips_DB_2018.txt", ";")
Line_List = Trips_df.LINEID.unique().tolist()

In [5]:
with open('Random_Forest_Results_2.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Route","Direction", "Mean_Absolute_Error_Train","Mean_Absolute_Error_Test", 
                     "Mean_Percentage_Error", "Mean_Absolute_Percentage_Error",
                     "Mean_Squared_Error", "Root_Mean_Squared_Error", "R_Square"])

In [ ]:
for Route in Line_List[32:]:
    df = pd.read_csv("~/data/Cleaned_By_Route_Header/route_{}.csv".format(Route))
    print("//--------------------------------------//")
    
    for direction in range(1,3):
        print("route:",Route,"\n","direction:", direction)
        final_df = Final_Clean(df, direction)
        Random_Forest(final_df)   

//--------------------------------------//
route: 13 
 direction: 1
Mean Absolute Error (train): 11.61029887698071
Mean Absolute Error (test): 12.31311598000338
Mean Percentage Error: -0.11682607008427705
Mean Absolute Percentage Error: 0.25764848091892173
Mean Squared Error: 431.7418953647264
Root Mean Squared Error: 20.778399730603088
R-square value: 0.766962599557631
route: 13 
 direction: 2
Mean Absolute Error (train): 12.078025362229203
Mean Absolute Error (test): 12.723108214121073
Mean Percentage Error: -0.11762055162559677
Mean Absolute Percentage Error: 0.26396000576569895
Mean Squared Error: 401.6238601209581
Root Mean Squared Error: 20.040555384543566
R-square value: 0.7237331475001981
//--------------------------------------//
route: 15B 
 direction: 1
Mean Absolute Error (train): 11.630199267149097
Mean Absolute Error (test): 12.765683049716612
Mean Percentage Error: -0.10929992191075945
Mean Absolute Percentage Error: 0.2607410246820513
Mean Squared Error: 409.94740228984

//--------------------------------------//
route: 42 
 direction: 1


In [5]:
for Route in Line_List[58:]:
    df = pd.read_csv("~/data/Cleaned_By_Route_Header/route_{}.csv".format(Route))
    print("//--------------------------------------//")
    
    for direction in range(1,3):
        print("route:",Route,"\n","direction:", direction)
        final_df = Final_Clean(df, direction)
        Random_Forest(final_df)   

//--------------------------------------//
route: 150 
 direction: 1
Mean Absolute Error (train): 11.062344847457814
Mean Absolute Error (test): 12.106672751890525
Mean Percentage Error: -0.08214788453336641
Mean Absolute Percentage Error: 0.22306412661620934
Mean Squared Error: 386.5227797504041
Root Mean Squared Error: 19.660182597076865
R-square value: 0.7486567804160456
route: 150 
 direction: 2
Mean Absolute Error (train): 11.059739250984611
Mean Absolute Error (test): 12.23081549895281
Mean Percentage Error: -0.10664421116560834
Mean Absolute Percentage Error: 0.2502842579108262
Mean Squared Error: 459.58688816636334
Root Mean Squared Error: 21.437977707012465
R-square value: 0.7006128148257191
//--------------------------------------//
route: 56A 
 direction: 1
Mean Absolute Error (train): 11.33821668355979
Mean Absolute Error (test): 12.438776400052737
Mean Percentage Error: -0.12004627725706803
Mean Absolute Percentage Error: 0.2742271556032283
Mean Squared Error: 355.77541481

OSError: [Errno 28] No space left on device

In [10]:
results = pd.read_csv("~/data/Model_Testing/Random_Forest_Results_2.csv")


In [12]:
results['Mean_Absolute_Error_Test'].mean()

12.641268937593289

In [ ]:
for Route in Line_List:
    df = pd.read_csv("~/data/Cleaned_By_Route_Header/route_{}.csv".format(Route))
    print("//--------------------------------------//")
    
    for direction in range(1,3):
        print("route:",Route,"\n","direction:", direction)
        final_df = Final_Clean(df, direction)
        Random_Forest(final_df)  

In [3]:
pd.read_csv("~/data/Model_Testing/Random_Forest_Results_2.csv")

,Route,Direction,Mean_Absolute_Error_Train,Mean_Absolute_Error_Test,Mean_Percentage_Error,Mean_Absolute_Percentage_Error,Mean_Squared_Error,Root_Mean_Squared_Error,R_Square
0,68,1,8.960011,9.619655,-0.096821,0.227003,263.246368,16.224869,0.775298
1,68,2,10.130505,10.859567,-0.112361,0.246257,355.673719,18.859314,0.685469
2,25B,1,12.095771,13.252714,-0.089149,0.226143,429.826271,20.732252,0.810621
3,25B,2,13.660799,14.864947,-0.101281,0.240120,596.506176,24.423476,0.711540
4,45A,1,8.920052,9.748322,-0.071881,0.204101,239.252236,15.467781,0.744367
...,...,...,...,...,...,...,...,...,...
117,150,2,11.055863,12.235453,-0.106871,0.250362,459.162636,21.428081,0.700889
118,150,1,11.062345,12.106673,-0.082148,0.223064,386.522780,19.660183,0.748657
119,150,2,11.059739,12.230815,-0.106644,0.250284,459.586888,21.437978,0.700613
120,56A,1,11.338217,12.438776,-0.120046,0.274227,355.775415,18.862010,0.757688


In [15]:
df_104 = pd.read_csv("~/data/Cleaned_By_Route_Header/route_104.csv")

In [27]:
df_83 = pd.read_csv("~/data/Cleaned_By_Route_Header/route_83.csv")


In [28]:
final_df = Final_Clean(df_83, 1)

In [32]:
X = pd.DataFrame(final_df[['DAY','HOUR','MONTH','STOPPOINTID','NEXT_STOPPOINTID',
                       'PEAK_HOUR', 'wind_speed', 'temp', 'humidity', 'weather_main']])

y = final_df.JOURNEY_DURATION

X = pd.get_dummies(X)

#test-train split 
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=1)

params = {'bootstrap': False,
 'max_depth': 50,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 64}

#creating and fitting the RF model and  Fit model on full dataset
rfc = RandomForestRegressor(**params).fit(X_train, Y_train)

#predictions from the training set
y_pred_rf_train = rfc.predict(X_train)

#predicitng from the test set 
y_pred_rf = rfc.predict(X_test)

#some evaluation metrics.
print('Mean Absolute Error (train):', metrics.mean_absolute_error(Y_train, y_pred_rf_train))
print('Mean Absolute Error (test):', metrics.mean_absolute_error(Y_test, y_pred_rf))
print('Mean Percentage Error:', np.mean((Y_test - y_pred_rf)/Y_test))
print('Mean Absolute Percentage Error:', metrics.mean_absolute_percentage_error(Y_test,y_pred_rf)) 
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred_rf))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred_rf)))
print('R-square value:', r2_score(Y_test,y_pred_rf))

Mean Absolute Error (train): 12.519164388043546
Mean Absolute Error (test): 13.648424096383541
Mean Percentage Error: -0.12236190961480148
Mean Absolute Percentage Error: 0.27837857849430886
Mean Squared Error: 417.7507374131832
Root Mean Squared Error: 20.43895147538599
R-square value: 0.6474326355705269


In [30]:
X = pd.DataFrame(final_df[['DAY','HOUR','MONTH','STOPPOINTID','NEXT_STOPPOINTID',
                       'PEAK_HOUR', 'wind_speed', 'temp', 'humidity', 'weather_main']])

y = final_df.JOURNEY_DURATION

X = pd.get_dummies(X)

#test-train split 
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [31]:
# Number of trees in random forest
n_estimators = [16,32,64,128]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [16, 32, 64, 128], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [32]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=2, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [16, 32, 64, 128]},
                   random_state=2, verbose=2)

In [33]:
rf_random.best_params_

{'n_estimators': 128,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': False}

In [34]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [35]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_train, Y_train)

Model Performance
Average Error: 11.0942 degrees.
Accuracy = 79.79%.


In [36]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [128, 150, 250, 500],
     'min_samples_split': [9,10,11],
     'min_samples_leaf': [3,4,5],
     'max_features': ['sqrt'],
     'max_depth': [80, 90, 100, 110, 120],
     'bootstrap': [False]
}

# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 180 candidates, totalling 540 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [False],
                         'max_depth': [80, 90, 100, 110, 120],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [9, 10, 11],
                         'n_estimators': [128, 150, 250, 500]},
             verbose=2)

In [37]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 90,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 500}

While having 500 estimators would improve our models, it does result in very large pickle files and as a result a lower amount may have to be used. We will keep all of the other best parameters as is.

In [38]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_train, Y_train)

Model Performance
Average Error: 10.6010 degrees.
Accuracy = 80.69%.


In [21]:
grid_search.best_params_

{'max_depth': 90, 'max_features': 'sqrt', 'n_estimators': 64}